In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

import pickle
import joblib
import os
from datetime import datetime

# Set style for visualizations
#plt.style.use('seaborn')
sns.set_palette("husl")



In [2]:
data = {'index': list(range(50)), 'value': np.random.rand(50)}
df = pd.DataFrame(data).set_index('index')

def safe_access(df, index):
    """Safely access a row in a DataFrame without causing a KeyError."""
    if index in df.index:
        return df.loc[index]
    else:
        print(f"Warning: Index {index} not found in DataFrame.")
        return None

# Example usage in your script
index_to_access = 24
result = safe_access(df, index_to_access)

if result is not None:
    # Proceed with processing result
    print(result)

class EnhancedLSTMForecaster:
    def __init__(self, sequence_length=24, n_features=None):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.model = None
        self.scaler = None
        self.history = None

    def create_sequences(self, X, y=None):
        """Create sequences for LSTM input with consistent lengths"""
        sequences_X = []
        sequences_y = []

        # Convert to numpy array if DataFrame
        if isinstance(X, pd.DataFrame):
            X = X.values
        if y is not None and isinstance(y, pd.Series):
            y = y.values

        # Create sequences ensuring consistent lengths
        for i in range(len(X) - self.sequence_length):
            seq_x = X[i:(i + self.sequence_length)]
            sequences_X.append(seq_x)

            if y is not None:
                sequences_y.append(y[i + self.sequence_length])

        sequences_X = np.array(sequences_X)

        if y is not None:
            sequences_y = np.array(sequences_y)
            # Ensure X and y have same number of samples
            min_len = min(len(sequences_X), len(sequences_y))
            return sequences_X[:min_len], sequences_y[:min_len]

        return sequences_X

    def build_model(self):
        """Build an enhanced LSTM model architecture"""
        model = Sequential([
            Bidirectional(LSTM(128, return_sequences=True),
                         input_shape=(self.sequence_length, self.n_features)),
            BatchNormalization(),
            Dropout(0.3),

            Bidirectional(LSTM(64, return_sequences=True)),
            BatchNormalization(),
            Dropout(0.3),

            LSTM(32),
            BatchNormalization(),
            Dropout(0.2),

            Dense(16, activation='relu'),
            BatchNormalization(),
            Dense(8, activation='relu'),
            Dense(1)
        ])

        optimizer = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

        self.model = model
        return model

    def train(self, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
        """Train the LSTM model with consistent sequence lengths"""
        # Create sequences
        print("Creating training sequences...")
        X_train_seq, y_train_seq = self.create_sequences(X_train, y_train)
        print(f"Training sequences shape: X={X_train_seq.shape}, y={y_train_seq.shape}")

        print("Creating validation sequences...")
        X_val_seq, y_val_seq = self.create_sequences(X_val, y_val)
        print(f"Validation sequences shape: X={X_val_seq.shape}, y={y_val_seq.shape}")

        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                mode='min'
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=0.00001,
                mode='min'
            )
        ]

        self.history = self.model.fit(
            X_train_seq, y_train_seq,
            validation_data=(X_val_seq, y_val_seq),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )

        return self.history

    def predict(self, X):
        """Make predictions with proper sequence handling"""
        # Create sequences without labels
        X_seq = self.create_sequences(X)
        predictions = self.model.predict(X_seq)

        # Create padded predictions array
        padded_predictions = np.full(len(X), np.nan)
        padded_predictions[self.sequence_length:len(X_seq) + self.sequence_length] = predictions.flatten()

        return padded_predictions

    def plot_training_history(self):
        """Plot training history"""
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(self.history.history['loss'], label='Training Loss')
        plt.plot(self.history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(self.history.history['mae'], label='Training MAE')
        plt.plot(self.history.history['val_mae'], label='Validation MAE')
        plt.title('Model MAE')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.legend()

        plt.tight_layout()
        plt.savefig('eda_plots/lstm_training_history.png')
        plt.close()

value    0.106426
Name: 24, dtype: float64


In [3]:
# Load and prepare data functions remain the same
def load_data(train_path, test_path):
    train_df = pd.read_excel(train_path)
    test_df = pd.read_excel(test_path)

    numeric_columns = ['Year', 'Month', 'Day', 'Hour', 'Load'] + \
                     [col for col in train_df.columns if 'Temp' in col or 'GHI' in col]
    for col in numeric_columns:
        if col in train_df.columns:
            train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
        if col in test_df.columns:
            test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

    return train_df, test_df

def create_datetime_features(df):
    """Create datetime features from relative Year (1,2,3), Month, Day, Hour columns"""
    base_year = 2020
    df['actual_year'] = base_year + df['Year'] - 1

    date_components = ['actual_year', 'Month', 'Day', 'Hour']
    for col in date_components:
        df[col] = df[col].astype(int)

    df['datetime'] = pd.to_datetime({
        'year': df['actual_year'],
        'month': df['Month'],
        'day': df['Day'],
        'hour': df['Hour']
    })

    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['quarter'] = df['datetime'].dt.quarter
    df['month'] = df['datetime'].dt.month
    df['hour'] = df['datetime'].dt.hour
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    df['day_of_year'] = df['datetime'].dt.dayofyear

    df['season'] = pd.cut(df['month'],
                         bins=[0, 3, 6, 9, 12],
                         labels=['Winter', 'Spring', 'Summer', 'Fall'])

    df = df.drop('actual_year', axis=1)

    return df

In [4]:
def perform_eda(train_df):
    """Perform Exploratory Data Analysis"""
    # Create directory for plots if it doesn't exist
    import os
    if not os.path.exists('eda_plots'):
        os.makedirs('eda_plots')

    # 1. Time Series Analysis
    plt.figure(figsize=(15, 8))
    plt.plot(train_df['datetime'], train_df['Load'])
    plt.title('Load Over Time')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('eda_plots/load_time_series.png')
    plt.close()

    # 2. Distribution Analysis
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    sns.histplot(train_df['Load'], bins=50)
    plt.title('Load Distribution')

    plt.subplot(2, 2, 2)
    sns.boxplot(x='season', y='Load', data=train_df)
    plt.title('Load by Season')

    plt.subplot(2, 2, 3)
    sns.boxplot(x='hour', y='Load', data=train_df)
    plt.title('Load by Hour')

    plt.subplot(2, 2, 4)
    sns.boxplot(x='dayofweek', y='Load', data=train_df)
    plt.title('Load by Day of Week')

    plt.tight_layout()
    plt.savefig('eda_plots/load_distributions.png')
    plt.close()

    # 3. Correlation Analysis
    temp_cols = [col for col in train_df.columns if 'Temp' in col]
    ghi_cols = [col for col in train_df.columns if 'GHI' in col]
    features_for_corr = ['Load'] + temp_cols + ghi_cols

    plt.figure(figsize=(12, 8))
    correlation = train_df[features_for_corr].corr()
    sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Heatmap')
    plt.tight_layout()
    plt.savefig('eda_plots/correlation_heatmap.png')
    plt.close()

    # Print summary statistics
    print("\nSummary Statistics for Load:")
    print(train_df['Load'].describe())

    # Temporal patterns
    print("\nAverage Load by Season:")
    print(train_df.groupby('season')['Load'].mean())

    print("\nAverage Load by Hour:")
    print(train_df.groupby('hour')['Load'].mean())

    # Return some key statistics for use in modeling
    return {
        'load_mean': train_df['Load'].mean(),
        'load_std': train_df['Load'].std(),
        'load_min': train_df['Load'].min(),
        'load_max': train_df['Load'].max()
    }

In [5]:
def prepare_features(df, is_training=True):
    """Prepare features for modeling"""
    # Calculate rolling means for temperature and GHI
    temp_cols = [col for col in df.columns if 'Temp' in col]
    ghi_cols = [col for col in df.columns if 'GHI' in col]

    # Sort by datetime to ensure correct rolling calculations
    df = df.sort_values('datetime')

    for col in temp_cols + ghi_cols:
        df[f'{col}_rolling_mean_24h'] = df[col].rolling(window=24, min_periods=1).mean()
        df[f'{col}_rolling_mean_7d'] = df[col].rolling(window=24*7, min_periods=1).mean()

    # Create lag features for Load (only for training data)
    if 'Load' in df.columns and is_training:
        df['Load_lag_1h'] = df['Load'].shift(1)
        df['Load_lag_24h'] = df['Load'].shift(24)
        df['Load_lag_7d'] = df['Load'].shift(24*7)

        # Rolling statistics for Load
        df['Load_rolling_mean_24h'] = df['Load'].rolling(window=24, min_periods=1).mean()
        df['Load_rolling_std_24h'] = df['Load'].rolling(window=24, min_periods=1).std()

    # Drop rows with NaN values created by lag features
    if is_training:
        df = df.dropna()

    return df

In [11]:
class LoadForecaster:
    def __init__(self):
        self.models = {
            'linear': LinearRegression(),
            'rf': RandomForestRegressor(n_estimators=100, random_state=42),
            'gbm': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'xgb': XGBRegressor(n_estimators=100, random_state=42),
            'lstm': None
        }
        self.scaler = StandardScaler()
        self.feature_cols = None

    def train_evaluate_models(self, X_train, y_train, X_val, y_val):
        results = {}
        self.feature_cols = X_train.columns

        # Scale the features
        X_train_scaled = pd.DataFrame(
            self.scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_val_scaled = pd.DataFrame(
            self.scaler.transform(X_val),
            columns=X_val.columns,
            index=X_val.index
        )

        # Train traditional models
        for name, model in self.models.items():
            if name != 'lstm':
                print(f"\nTraining {name} model...")
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_val_scaled)
                results[name] = self.calculate_metrics(y_val, y_pred)
                self.plot_predictions(y_val, y_pred, name)

        # Train LSTM model
        print("\nTraining LSTM model...")
        lstm_forecaster = EnhancedLSTMForecaster(
            sequence_length=24,
            n_features=X_train.shape[1]
        )
        lstm_forecaster.build_model()

        # Make sure data is properly aligned before sequence creation
        X_train_seq = X_train_scaled.copy()
        y_train_seq = y_train.copy()
        X_val_seq = X_val_scaled.copy()
        y_val_seq = y_val.copy()

        # Print shapes before sequence creation
        print(f"Before sequence creation - X_train: {X_train_seq.shape}, y_train: {y_train_seq.shape}")

        history = lstm_forecaster.train(
            X_train_seq, y_train_seq,
            X_val_seq, y_val_seq,
            epochs=100,
            batch_size=32
        )

        lstm_predictions = lstm_forecaster.predict(X_val_seq)
        valid_predictions = lstm_predictions[~np.isnan(lstm_predictions)]
        valid_targets = y_val.iloc[len(y_val)-len(valid_predictions):]

        results['lstm'] = self.calculate_metrics(valid_targets, valid_predictions)
        self.plot_predictions(valid_targets, valid_predictions, 'lstm')

        lstm_forecaster.plot_training_history()

        return results

    def calculate_metrics(self, y_true, y_pred):
        return {
            'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
            'MAE': mean_absolute_error(y_true, y_pred),
            'R2': r2_score(y_true, y_pred)
        }

    def plot_predictions(self, y_true, y_pred, model_name):
        plt.figure(figsize=(10, 6))
        plt.plot(y_true.values[:100], label='Actual')
        plt.plot(y_pred[:100], label='Predicted')
        plt.title(f'{model_name.upper()} Model - Actual vs Predicted (First 100 points)')
        plt.legend()
        plt.savefig(f'eda_plots/{model_name}_predictions.png')
        plt.close()


In [14]:
def save_trained_models(forecaster, base_path='saved_models'):
      """Save trained models from a LoadForecaster instance.

      Args:
          forecaster (LoadForecaster): Instance of LoadForecaster with trained models
          base_path (str): Base directory to save the models
      """
      # Create timestamp for versioning
      timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

      # Create base directory if it doesn't exist
      model_path = os.path.join(base_path, timestamp)
      os.makedirs(model_path, exist_ok=True)

      # Save feature columns
      if forecaster.feature_cols is not None:
          joblib.dump(forecaster.feature_cols, os.path.join(model_path, 'feature_cols.joblib'))

      # Save scaler
      joblib.dump(forecaster.scaler, os.path.join(model_path, 'scaler.joblib'))

      # Save traditional ML models
      for name, model in forecaster.models.items():
          if name != 'lstm' and model is not None:
              joblib.dump(model, os.path.join(model_path, f'{name}_model.joblib'))

      # Save LSTM model if it exists
      if forecaster.models['lstm'] is not None:
          lstm_path = os.path.join(model_path, 'lstm_model')
          os.makedirs(lstm_path, exist_ok=True)

          # Save model architecture and weights
          model_json = forecaster.models['lstm'].model.to_json()
          with open(os.path.join(lstm_path, 'model_architecture.json'), 'w') as f:
              f.write(model_json)

          # Save weights
          forecaster.models['lstm'].model.save_weights(os.path.join(lstm_path, 'model_weights.h5'))

          # Save sequence length and other LSTM-specific parameters
          lstm_params = {
              'sequence_length': forecaster.models['lstm'].sequence_length,
              'n_features': forecaster.models['lstm'].n_features
          }
          joblib.dump(lstm_params, os.path.join(lstm_path, 'lstm_params.joblib'))

      print(f"Models saved successfully in {model_path}")
      return model_path

In [15]:
def main():
    print("Loading data...")
    train_df, test_df = load_data('training.xlsx', 'testing.xlsx')

    print("Creating datetime features...")
    train_df = create_datetime_features(train_df)
    test_df = create_datetime_features(test_df)

    print("Performing EDA...")
    perform_eda(train_df)

    print("Preparing features...")
    train_df = prepare_features(train_df, is_training=True)
    test_df = prepare_features(test_df, is_training=False)

    # Sort the dataframe by datetime to ensure proper sequence creation
    train_df = train_df.sort_values('datetime')

    # Calculate the split point
    train_size = int(len(train_df) * 0.8)

    # Prepare features for modeling
    feature_cols = [col for col in train_df.columns if col not in
                   ['datetime', 'Load', 'Year', 'Month', 'Day', 'Hour', 'season']]

    # Create X and y before splitting
    X = train_df[feature_cols]
    y = train_df['Load']

    # Split the data
    X_train = X[:train_size]
    y_train = y[:train_size]
    X_val = X[train_size:]
    y_val = y[train_size:]

    # Ensure indices are reset and aligned
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)

    print(f"Training set shapes - X: {X_train.shape}, y: {y_train.shape}")
    print(f"Validation set shapes - X: {X_val.shape}, y: {y_val.shape}")

    print("Training models...")
    forecaster = LoadForecaster()
    results = forecaster.train_evaluate_models(X_train, y_train, X_val, y_val)

    # Save the trained models
    print("Saving models...")
    model_path = save_trained_models(forecaster)
    print(f"Models saved to: {model_path}")

    print("\nModel Performance Metrics:")
    for model_name, metrics in results.items():
        print(f"\n{model_name.upper()} Model:")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value:.4f}")

if __name__ == "__main__":
    main()



Loading data...
Creating datetime features...
Performing EDA...

Summary Statistics for Load:
count    17544.000000
mean      2154.130529
std        437.248479
min       1027.000000
25%       1847.000000
50%       2085.000000
75%       2392.000000
max       4397.000000
Name: Load, dtype: float64

Average Load by Season:
season
Winter    2016.964088
Spring    1931.864469
Summer    2488.539176
Fall      2174.502038
Name: Load, dtype: float64

Average Load by Hour:
hour
0     2039.465116
1     1951.441860
2     1885.556772
3     1851.239398
4     1859.811218
5     1922.331053
6     2023.117647
7     2114.181943
8     2124.704514
9     2053.954856
10    1969.744186
11    1920.242134
12    1917.149111
13    1959.822161
14    2047.986320
15    2187.968536
16    2352.477428
17    2535.569083
18    2666.336525
19    2687.476060
20    2626.852257
21    2504.279070
22    2335.606019
23    2161.819425
Name: Load, dtype: float64
Preparing features...
Training set shapes - X: (13900, 41), y: (13900